In [1]:
from datetime import datetime
import datetime as dt
import os
from operator import itemgetter
import shutil 
import pickle
import time
import json

from django.contrib.gis.geos.geometry import GEOSGeometry
from osgeo import osr
from osgeo import gdal
from pyproj import Proj
from fiona.crs import from_epsg
from fiona.crs import to_string
from sklearn.cluster import KMeans
import geopandas as gpd
from affine import Affine
import xarray as xr
import numpy as np
from rasterio.features import rasterize
from shapely.geometry import shape
import fiona
import glob
from dask.distributed import Client
from datacube.drivers.netcdf import write_dataset_to_netcdf
from datacube.api import GridWorkflow
import rasterio
from rasterio.mask import mask

from madmex.wrappers import gwf_query
from madmex.util.xarray import to_float, to_int
from madmex.models import PredictObject, TrainClassification
from madmex.models import Region

2020-02-25 17:00:03,643 - datacube.drivers.driver_cache - driver_cache safe_load: Failed to resolve driver datacube.plugins.index::s3aio_index
2020-02-25 17:00:03,645 - datacube.drivers.driver_cache - driver_cache safe_load: Error was: UndefinedEnvironmentName("'extra' does not exist in evaluation environment.",)
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:44116 remote=tcp://127.0.0.1:35959>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:44118 remote=tcp://127.0.0.1:35959>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:44120 remote=tcp://127.0.0.1:35959>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:44122 remote=tcp://127.0.0.1:35959>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:44124 remote=tcp://127.0.0.1:35959>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP 

In [2]:
client = Client(n_workers=5,memory_limit='110GB', threads_per_worker=1)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:35959 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 550.00 GB


In [4]:
client.restart()

Client Scheduler: tcp://127.0.0.1:35959 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 550.00 GB


In [5]:
region = 'cultivos_2019_simplify'
name_of_product = 's2_l2a_10m_scl_mexico'
begin = '2018-01-01'
end = '2019-12-31'
gwf_kwargs = {'region': region, 'product': name_of_product, 'begin': begin, 'end':end}
iterable = gwf_query(**gwf_kwargs)

In [6]:
lista_iter = list(iterable)

In [7]:
len(lista_iter)

53

In [8]:
[elem[0] for elem in lista_iter]

[(47, -35),
 (47, -37),
 (48, -37),
 (47, -36),
 (48, -36),
 (40, -31),
 (39, -31),
 (38, -30),
 (39, -30),
 (40, -30),
 (39, -29),
 (38, -29),
 (40, -29),
 (38, -28),
 (40, -28),
 (38, -27),
 (39, -28),
 (39, -27),
 (38, -26),
 (39, -26),
 (40, -33),
 (40, -32),
 (39, -32),
 (37, -29),
 (40, -34),
 (41, -33),
 (42, -33),
 (41, -32),
 (42, -32),
 (41, -31),
 (42, -31),
 (43, -33),
 (44, -34),
 (43, -35),
 (45, -34),
 (44, -35),
 (45, -37),
 (43, -34),
 (44, -37),
 (43, -36),
 (45, -36),
 (45, -35),
 (44, -36),
 (42, -34),
 (41, -34),
 (37, -27),
 (37, -26),
 (37, -28),
 (41, -30),
 (46, -35),
 (46, -34),
 (46, -36),
 (46, -37)]

# 1) Se elige el primer dc_tile de lista_iter para la generación de segmentos clusterizados

In [9]:
date_today = datetime.today().strftime("%m-%d-%Y")

In [14]:
index=4

In [15]:
var=lista_iter[index]

In [16]:
dc_tile = '%d_%d' % (var[0][0], var[0][1])

In [17]:
dc_tile

'48_-36'

In [20]:
date_today

'02-23-2020'

In [21]:
sr_0 = GridWorkflow.load(var[1],dask_chunks={'x': 1600, 'y': 1600,'time': 60})

In [22]:
sr_0.attrs['geobox'] = var[1].geobox

In [23]:
sr_0 = client.persist(sr_0)

**110gb is enough**

In [24]:
sr_0 = sr_0.apply(func=to_float, keep_attrs=True)

In [25]:
sr_1 = sr_0.where(sr_0.pixel_qa.isin([2,4,5,6,7,8,11]))

In [26]:
sr_1['ndvi'] = ((sr_1.nir - sr_1.red) / (sr_1.nir + sr_1.red)) * 10000

In [27]:
sr_1['ndvi'].attrs['nodata'] = 0

In [28]:
sr_1['gndvi'] = ((sr_1.nir - sr_1.green) / (sr_1.nir + sr_1.green)) * 10000

In [29]:
sr_1['gndvi'].attrs['nodata'] = 0

In [30]:
sr_1['avi'] = ((sr_1.nir*(1-sr_1.red))*(sr_1.nir-sr_1.red))**(1/3)

In [31]:
sr_1['avi'].attrs['nodata'] = 0

In [32]:
sr_1 = client.persist(sr_1)

# 2) Cálculo de features

In [33]:
ndvi_mean_resampled = sr_1.ndvi.resample(time='M').mean()

In [34]:
ndvi_mean_resampled = client.persist(ndvi_mean_resampled)

In [35]:
ndvi_mean_resampled_drop_na = ndvi_mean_resampled.dropna('time',how='all')

In [36]:
gndvi_mean_resampled = sr_1.gndvi.resample(time='M').mean()

In [37]:
gndvi_mean_resampled = client.persist(gndvi_mean_resampled)

In [38]:
gndvi_mean_resampled_drop_na = gndvi_mean_resampled.dropna('time',how='all')

In [39]:
avi_mean_resampled = sr_1.avi.resample(time='M').mean()

In [40]:
avi_mean_resampled = client.persist(avi_mean_resampled)

In [41]:
avi_mean_resampled_drop_na = avi_mean_resampled.dropna('time',how='all')

In [42]:
combined = xr.merge([ndvi_mean_resampled_drop_na,
                     gndvi_mean_resampled_drop_na,
                     avi_mean_resampled_drop_na])

In [43]:
combined = client.persist(combined)

In [44]:
features = list(combined.data_vars)
s = '-'
features_string = s.join(features)

In [45]:
path_result = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/' + features_string + '/'

In [46]:
path_result_dc_tile = os.path.join(path_result, date_today 
                                   + '/' + dc_tile + '/')

In [47]:
path_result_dc_tile

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-23-2020/48_-36/'

In [48]:
if not os.path.exists(path_result_dc_tile):
    os.makedirs(path_result_dc_tile)

In [49]:
nc_filename = path_result_dc_tile + date_today + '_combined_' + dc_tile + '.nc'

In [50]:
combined.attrs = sr_0.attrs

In [51]:
combined.coords['time'].attrs = sr_0.coords['time'].attrs
combined.coords['x'].attrs = sr_0.coords['x'].attrs
combined.coords['y'].attrs = sr_0.coords['y'].attrs

In [52]:
#write_dataset_to_netcdf(combined.compute(),nc_filename)

# 3) Leer segmentación

In [53]:
geom = GEOSGeometry(json.dumps(sr_0.geobox.geographic_extent.json))

In [54]:
seg_name = 'seg_mex_s2_10m_scl_2018_2019'

In [55]:
qs = PredictObject.objects.filter(the_geom__contained=geom,segmentation_information__name=seg_name)

In [56]:
path_seg = qs[0].path

In [58]:
path_seg

'/shared_volume/segmentation_antares3_results/bis_seg_mex_s2_10m_scl_2018_2019_sentinel_2_48_-36_2018_2019.shp'

In [59]:
with fiona.open(path_seg) as src:
    fc = list(src)
    crs = to_string(src.crs)

In [60]:
fc_subset=((x['properties']['id'], x['geometry']) for x in fc)

In [61]:
fc_sorted = sorted(fc_subset, key=itemgetter(0))

**First position of fc_sorted is id, second position is geometry**


In [62]:
iterable_zip = zip([k[1] for k in fc_sorted], [k[0] for k in fc_sorted])

In [63]:
aff = Affine(*list(sr_0.affine)[0:6])

In [64]:
dimensions_dataset = list(sr_0.coords)
list_dimensions = [x for x in dimensions_dataset if x != 'time']
lambda_function = lambda l_netcdf,l_test: l_netcdf[0] if l_netcdf[0] in l_test else l_netcdf[1]
xdim = lambda_function(list_dimensions,['x','longitude'])
ydim = lambda_function(list_dimensions,['y','latitude'])

In [65]:
fc_raster = rasterize(iterable_zip, transform=aff,
                      out_shape=(sr_0.sizes[ydim], sr_0.sizes[xdim]),
                      dtype='float64', fill=np.nan)

In [66]:
fc_raster = fc_raster.astype(int)

In [67]:
fc_dataarray = xr.DataArray(fc_raster, dims=[ydim, xdim], name='features_id')

# 4) Construcción de dataframes para pasar al kmeans

In [68]:
mask_file='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/madmex_mascara_s2_2018_agricultura_pastizal_Veracruz_una_clase.tif'

Abrir máscara, recortar máscara a tamaño de extent de dc tile

In [69]:
crs_dict = {'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'a':6378137,
 'b':6378136.027241431,
 'wktext': True}

#+ellps=WGS84 +datum=WGS84

In [70]:
geometry_extent = sr_0.geobox.extent.json

In [139]:
geometry_extent

{'type': 'Polygon',
 'coordinates': [[(3378120.0, 676020.0),
   (3378120.0, 626000.0),
   (3428140.0, 626000.0),
   (3428140.0, 676020.0),
   (3378120.0, 676020.0)]]}

In [294]:
#xmin,ymin,xmax,ymax=sr_0.geobox.extent.boundingbox

In [295]:
#(xmin,ymin,xmax,ymax)

In [245]:
#mask_crop_file='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/' + date_today + '_' + dc_tile +'_gdal_translate.tif'

In [246]:
#mask_crop_file

In [247]:
#gdal.Translate(mask_crop_file, ds, options=['COMPRESS=LZW'])

In [180]:
#outname='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/madmex_mascara_s2_2018_agricultura_pastizal_Veracruz_una_clase_48_-36_python.tif'

In [181]:
#shp='/LUSTRE/MADMEX/shapefiles/datacube_tiles/datacube_tile_48_-36.shp'

In [193]:
geom_extent_schema={'properties':{'id':'int'},
                    'geometry': 'Polygon'}

In [204]:
layer_name = 'geom_extent_' + dc_tile

In [205]:
geom_extent_file = path_result + '/' + date_today + '/' + dc_tile + '/' + layer_name + '.shp'

In [206]:
geom_extent_file

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi//02-23-2020/48_-36/geom_extent_48_-36.shp'

In [207]:
with fiona.open(geom_extent_file,'w',
                driver='ESRI Shapefile',
                crs=crs_dict,
                layer=layer_name,
                schema=geom_extent_schema) as dst:
    dst.write({'properties':{'id':0},
               'geometry': geometry_extent})
    

In [253]:
mask_crop = gdal.Warp('', mask_file, format = 'VRT',
                    cutlineDSName=geom_extent_file,
                    cutlineLayer = layer_name,
                    cropToCutline=True)

In [254]:
arr=mask_crop.ReadAsArray()

In [255]:
arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=uint8)

In [257]:
#file_example='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/madmex_mascara_s2_2018_agricultura_pastizal_Veracruz_una_clase_48_-36_with_geom_extent.tif'

In [258]:
#with rasterio.open(file_example) as src:
#    arr2 = src.read()

In [259]:
#arr2

In [141]:
#with rasterio.open(mask_file) as mask_src:
#    mask_src_crop, out_transform = mask(mask_src, shapes=[geometry_extent], crop=True)
#    mask_meta = mask_src.meta.copy()

In [260]:
#mask_meta

In [261]:
#mask_src_crop.shape

In [262]:
#out_transform

In [263]:
combined.ndvi.shape

(12, 5002, 5002)

In [264]:
arr.shape

(5002, 5002)

**step for verification**

In [265]:
#mask_crop_file='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/madmex_mascara_s2_2018_agricultura_pastizal_Veracruz_una_clase_crop_' + date_today + '_' + dc_tile +'_1st_crop.tif'

In [266]:
#mask_crop_file

In [267]:
#mask_meta.update(width=mask_src_crop.shape[2],
#                 height=mask_src_crop.shape[1],
#                 transform=out_transform,
#                 compress='lzw')

In [268]:
#mask_meta

In [269]:
#with rasterio.open(mask_crop_file,'w',**mask_meta) as dst:
#    dst.write(mask_src_crop)

**Need to resize mask_src_crop to shape of combined filling with 0 if needed**

In [270]:
#extent_rasterize = rasterize([geometry_extent], transform=aff,
#                      out_shape=(sr_0.sizes[ydim], sr_0.sizes[xdim]),
#                      dtype='float64', fill=np.nan,
#                      default_value=0)

In [271]:
#extent_rasterize

In [272]:
#extent_rasterize.shape

In [273]:
#mask_src_crop

In [274]:
#sr_0.affine

In [275]:
#out_transform

In [276]:
#mask_meta.update(width=combined.ndvi.shape[2],
#                 height=combined.ndvi.shape[1],
#                 transform=out_transform,
#                 compress='lzw')

In [277]:
#mask_meta

**Hay que generar un archivo por cada dc tile procesado (entonces añadir al siguiente mask_crop_file el dc_tile y el día quizas?)**

In [278]:
#mask_crop_file='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/madmex_mascara_s2_2018_agricultura_pastizal_Veracruz_una_clase_crop_' + date_today + '_' + dc_tile +'.tif'

In [279]:
#mask_crop_file

In [280]:
#with rasterio.open(mask_crop_file,'w',**mask_meta) as dst:
#    dst.write(mask_src_crop)
    

In [281]:
#mask_src_crop = rasterio.open(mask_crop_file)

In [282]:
#mask_src_crop.shape

In [283]:
#with rasterio.open(mask_crop_file,'r') as src:
#    array = src.read()
#    combined_masked = combined.where(array==1)

In [284]:
combined_masked = combined.where(arr==1)

In [285]:
combined_masked

<xarray.Dataset>
Dimensions:  (time: 12, x: 5002, y: 5002)
Coordinates:
  * time     (time) datetime64[ns] 2018-10-31 2018-11-30 ... 2019-09-30
  * y        (y) float64 6.76e+05 6.76e+05 6.76e+05 ... 6.26e+05 6.26e+05
  * x        (x) float64 3.378e+06 3.378e+06 3.378e+06 ... 3.428e+06 3.428e+06
Data variables:
    ndvi     (time, y, x) float64 dask.array<chunksize=(1, 1600, 1600), meta=np.ndarray>
    gndvi    (time, y, x) float64 dask.array<chunksize=(1, 1600, 1600), meta=np.ndarray>
    avi      (time, y, x) float64 dask.array<chunksize=(1, 1600, 1600), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...
    geobox:   GeoBox(Geometry({'type': 'Polygon', 'coordinates': [[(-93.70555...

In [286]:
combined = xr.merge([fc_dataarray,combined_masked])

In [287]:
combined = client.persist(combined)

In [288]:
combined = combined.stack(z=(xdim, ydim))
combined = combined.reset_index('z')
combined = combined.drop([xdim,ydim])

In [289]:
combined

<xarray.Dataset>
Dimensions:      (time: 12, z: 25020004)
Coordinates:
  * time         (time) datetime64[ns] 2018-10-31 2018-11-30 ... 2019-09-30
Dimensions without coordinates: z
Data variables:
    features_id  (z) int64 466825 466825 466825 466825 ... 690 690 690 690
    ndvi         (time, z) float64 dask.array<chunksize=(1, 2000800), meta=np.ndarray>
    gndvi        (time, z) float64 dask.array<chunksize=(1, 2000800), meta=np.ndarray>
    avi          (time, z) float64 dask.array<chunksize=(1, 2000800), meta=np.ndarray>

In [290]:
combined = combined.where((np.isfinite(combined['features_id'])), drop=True)

In [291]:
combined = client.persist(combined)

In [292]:
df = combined.to_dataframe()

In [296]:
df.head()

features_id  ndvi  gndvi  avi
time       z                               
2018-10-31 0     466825.0   NaN    NaN  NaN
           1     466825.0   NaN    NaN  NaN
           2     466825.0   NaN    NaN  NaN
           3     466825.0   NaN    NaN  NaN
           4     466825.0   NaN    NaN  NaN

In [297]:
del sr_0
del ndvi_mean_resampled_drop_na
del gndvi_mean_resampled_drop_na
del avi_mean_resampled_drop_na

In [ ]:
client.restart()

In [299]:
df=df.reset_index('time')

In [300]:
df2 = df.groupby(['time','features_id'],as_index=False).agg({features[0]: 'median',
                                                             features[1]: 'median',
                                                             features[2]: 'median'})

In [301]:
df2.index = df2['time']

In [302]:
df2.drop(['time'], axis=1, inplace=True)

In [303]:
#df_filename = path_result_dc_tile + date_today + '_df' + '.csv'

In [304]:
#df2.to_csv(df_filename)

In [305]:
times_df = [datetime.date(d).strftime('%Y-%m-%d') for d in df2.index.get_level_values('time').unique()]

In [306]:
df3 = df2.loc[times_df[0]]

In [307]:
df3.rename(columns={features[0]: features[0]+'_' + times_df[0],
                    features[1]: features[1]+'_' +times_df[0],
                    features[2]: features[2]+'_' +times_df[0]},
                 inplace=True)

In [308]:
df3=df3.reset_index('time')

In [309]:
df3.drop(['time'],axis=1,inplace=True)

In [310]:
for k in range(1,len(times_df)):
    df_subset = df2.loc[times_df[k]]
    df_subset.dropna(how='all', inplace=True)
    df_subset.rename(columns={features[0]: features[0]+'_' + times_df[k],
                        features[1]: features[1]+'_' +times_df[k],
                        features[2]: features[2]+'_' +times_df[k]},
                     inplace=True)
    df_subset=df_subset.reset_index('time')
    df_subset.drop(['time'],axis=1,inplace=True)
    df3 = df3.join(df_subset.set_index('features_id'), on='features_id')

In [311]:
len(df3.index)

467521

In [312]:
sum(df3['ndvi_2018-10-31'].isna())

432489

In [313]:
df3.head()

,features_id,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,avi_2018-12-31,...,avi_2019-06-30,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30
0,0.0,8439.402324,6922.295582,NaN,5971.509841,4859.299317,NaN,8547.066321,7241.363867,NaN,...,NaN,7093.633141,6004.498174,NaN,7748.516855,6484.989277,NaN,7225.806452,6341.689879,NaN
1,1.0,8690.660875,7527.405603,NaN,6971.499318,5628.418554,NaN,8717.252562,7636.720017,NaN,...,NaN,7914.011068,6758.943019,NaN,8237.472973,7057.782109,NaN,7555.718834,6995.871705,NaN
2,2.0,8429.524510,7200.217785,NaN,6291.282883,5363.265423,NaN,8284.274608,7175.785422,0.0,...,NaN,7681.365385,6322.218995,NaN,8424.945062,7016.830319,NaN,6686.559119,5964.226222,NaN
3,3.0,8781.037937,7624.985279,NaN,5476.743128,4550.230200,NaN,8802.999202,7678.782035,NaN,...,NaN,8507.628789,7333.131917,NaN,8120.352378,6959.798847,NaN,7714.394223,7033.605643,NaN
4,4.0,8715.166999,7599.370051,NaN,5781.876108,4975.492300,NaN,8777.576776,7639.982100,NaN,...,NaN,8510.226214,7361.318835,NaN,7125.091855,6095.316028,NaN,7230.390506,6480.530981,NaN


In [316]:
os.remove(geom_extent_file)
os.remove(path_result + '/' + date_today + '/' + dc_tile + '/' + layer_name + '.dbf')
os.remove(path_result + '/' + date_today + '/' + dc_tile + '/' + layer_name + '.shx')
os.remove(path_result + '/' + date_today + '/' + dc_tile + '/' + layer_name + '.cpg')
os.remove(path_result + '/' + date_today + '/' + dc_tile + '/' + layer_name + '.prj')

**Join df with fc in geopandas dataframe form**

In [427]:
#crs_dict = {'proj': 'lcc',
# 'lat_1': 17.5,
# 'lat_2': 29.5,
# 'lat_0': 12,
# 'lon_0': -102,
# 'x_0': 2500000,
# 'y_0': 0,
# 'datum': 'WGS84',
# 'units': 'm',
# 'no_defs': True,
# 'wktext': True}

In [317]:
gdf = gpd.GeoDataFrame.from_features([feature for feature in fc], 
                                      crs = crs_dict)


In [318]:
gdf = gdf.rename(columns={'id':'features_id'})

In [319]:
gdf_df_join = gdf.merge(df3, on='features_id', how='left')

In [320]:
gdf_df_join.head()

,geometry,features_id,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,...,avi_2019-06-30,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30
0,"POLYGON ((3378350.000 676020.000, 3378370.000 ...",467325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"POLYGON ((3378370.000 676020.000, 3378450.000 ...",467404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"POLYGON ((3379000.000 676020.000, 3379080.000 ...",467405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"POLYGON ((3379120.000 676020.000, 3379160.000 ...",467505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"POLYGON ((3379260.000 676020.000, 3379290.000 ...",467266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [321]:
len(gdf_df_join.index)

467521

In [322]:
sum(gdf_df_join['ndvi_2018-10-31'].isna())

432489

In [323]:
features_string

'ndvi-gndvi-avi'

In [324]:
gdf_df_join_reduced = gdf_df_join.drop(['features_id','geometry'],axis=1)

In [325]:
gdf_df_join_reduced.head()

,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,avi_2018-12-31,ndvi_2019-01-31,...,avi_2019-06-30,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [326]:
gdf_df_join_reduced.dropna(how='all', inplace=True)

In [327]:
gdf_df_join_reduced.head()

,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,avi_2018-12-31,ndvi_2019-01-31,...,avi_2019-06-30,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30
218053,8304.134548,7338.635668,NaN,7465.388711,6530.973451,NaN,7237.011477,6363.565164,NaN,8052.283826,...,NaN,7689.967486,6546.795110,NaN,7747.756800,6576.106806,NaN,7439.024390,6435.643564,NaN
218067,8133.212506,6894.308943,NaN,7690.610444,6666.666667,NaN,6896.273016,6029.897847,NaN,7395.264117,...,NaN,7381.496721,6107.223550,NaN,7924.126136,6654.759634,NaN,7442.273535,6275.456645,NaN
218071,7718.072476,6568.695744,NaN,7706.344546,6523.445696,NaN,7059.499921,6117.241135,NaN,7132.002012,...,NaN,7078.505184,5874.291074,NaN,7434.294687,6177.946374,NaN,7663.666181,6467.548990,NaN
218095,8247.998059,6930.722011,NaN,6778.811739,5661.469933,NaN,7870.432668,6739.181743,NaN,7293.343087,...,NaN,7704.398192,6520.507645,NaN,7534.888059,6295.096037,NaN,7674.641148,6522.172397,NaN
218455,7817.393399,6635.725606,NaN,3884.085815,3013.205282,NaN,6999.579138,6127.348506,NaN,7129.977461,...,NaN,7477.898665,6416.855145,NaN,5934.246852,4971.680971,NaN,7060.751197,6129.647283,NaN


In [328]:
len(gdf_df_join_reduced.index)

36338

In [329]:
t_keep_rows = gdf_df_join_reduced.index.values

In [330]:
gdf_df_join_reduced2 = gdf_df_join.loc[gdf_df_join.index.isin(t_keep_rows)]

In [331]:
gdf_df_join_reduced2.head()

,geometry,features_id,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,...,avi_2019-06-30,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30
218053,"POLYGON ((3378400.000 652720.000, 3378420.000 ...",249459,8304.134548,7338.635668,NaN,7465.388711,6530.973451,NaN,7237.011477,6363.565164,...,NaN,7689.967486,6546.795110,NaN,7747.756800,6576.106806,NaN,7439.024390,6435.643564,NaN
218067,"POLYGON ((3378260.000 652800.000, 3378270.000 ...",249361,8133.212506,6894.308943,NaN,7690.610444,6666.666667,NaN,6896.273016,6029.897847,...,NaN,7381.496721,6107.223550,NaN,7924.126136,6654.759634,NaN,7442.273535,6275.456645,NaN
218071,"POLYGON ((3378140.000 652640.000, 3378160.000 ...",249360,7718.072476,6568.695744,NaN,7706.344546,6523.445696,NaN,7059.499921,6117.241135,...,NaN,7078.505184,5874.291074,NaN,7434.294687,6177.946374,NaN,7663.666181,6467.548990,NaN
218095,"POLYGON ((3378130.000 652600.000, 3378160.000 ...",249189,8247.998059,6930.722011,NaN,6778.811739,5661.469933,NaN,7870.432668,6739.181743,...,NaN,7704.398192,6520.507645,NaN,7534.888059,6295.096037,NaN,7674.641148,6522.172397,NaN
218455,"POLYGON ((3378120.000 652520.000, 3378150.000 ...",248676,7817.393399,6635.725606,NaN,3884.085815,3013.205282,NaN,6999.579138,6127.348506,...,NaN,7477.898665,6416.855145,NaN,5934.246852,4971.680971,NaN,7060.751197,6129.647283,NaN


In [332]:
len(gdf_df_join_reduced2)

36338

# 5) Unsupervised clustering with Kmeans

**Número de clústers depende del número de cultivos presentes en el dc_tile** 

**Revisar credenciales de db**

In [333]:
var[1].geobox.geographic_extent.json

{'type': 'Polygon',
 'coordinates': [[(-93.70555365678503, 17.80907660608144),
   (-93.73275743058925, 17.360420769651387),
   (-93.2629666905944, 17.333807435148785),
   (-93.23422453278889, 17.78235440562819),
   (-93.70555365678503, 17.80907660608144)]]}

In [334]:
training_set = 'cultivos2019_inecol'

In [335]:
qs_t = TrainClassification.objects.filter(train_object__the_geom__contained=geom,
                                        training_set=training_set).prefetch_related('interpret_tag')

In [336]:
l_tag_ids = []

In [337]:
for element in qs_t:
    l_tag_ids.append(element.interpret_tag.id)

In [338]:
nclusters = len(np.unique(l_tag_ids))

In [339]:
nclusters

2

In [340]:
kmeans = KMeans(n_clusters=nclusters, n_jobs=10)

In [341]:
features_for_kmeans = [column for column in list(gdf_df_join_reduced2.columns) if column not in ['geometry','features_id']]

In [342]:
features_for_kmeans

['ndvi_2018-10-31',
 'gndvi_2018-10-31',
 'avi_2018-10-31',
 'ndvi_2018-11-30',
 'gndvi_2018-11-30',
 'avi_2018-11-30',
 'ndvi_2018-12-31',
 'gndvi_2018-12-31',
 'avi_2018-12-31',
 'ndvi_2019-01-31',
 'gndvi_2019-01-31',
 'avi_2019-01-31',
 'ndvi_2019-02-28',
 'gndvi_2019-02-28',
 'avi_2019-02-28',
 'ndvi_2019-03-31',
 'gndvi_2019-03-31',
 'avi_2019-03-31',
 'ndvi_2019-04-30',
 'gndvi_2019-04-30',
 'avi_2019-04-30',
 'ndvi_2019-05-31',
 'gndvi_2019-05-31',
 'avi_2019-05-31',
 'ndvi_2019-06-30',
 'gndvi_2019-06-30',
 'avi_2019-06-30',
 'ndvi_2019-07-31',
 'gndvi_2019-07-31',
 'avi_2019-07-31',
 'ndvi_2019-08-31',
 'gndvi_2019-08-31',
 'avi_2019-08-31',
 'ndvi_2019-09-30',
 'gndvi_2019-09-30',
 'avi_2019-09-30']

In [343]:
kmeans.fit(gdf_df_join_reduced2[features_for_kmeans].fillna(0).values)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=10, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

**Write model fit**

In [344]:
model_filename = path_result_dc_tile + date_today + '_model_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi' + '.pkl'

In [345]:
model_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-23-2020/48_-36/02-23-2020_model_48_-36_nclusters_2_aoi.pkl'

In [346]:
pickle.dump(kmeans, open(model_filename, "wb"))

**Predict with model**

In [347]:
model_filename = path_result_dc_tile + date_today + '_model_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi' + '.pkl'

In [348]:
model_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-23-2020/48_-36/02-23-2020_model_48_-36_nclusters_2_aoi.pkl'

In [349]:
kmeans = pickle.load(open(model_filename, "rb"))

**Predict**

In [350]:
preds = kmeans.predict(gdf_df_join_reduced2[features_for_kmeans].fillna(0).values)

In [351]:
gdf_df_clusters = gdf_df_join_reduced2.drop(features_for_kmeans,axis=1)


In [352]:
gdf_df_clusters['preds'] = preds

In [353]:
gdf_df_clusters.head()

,geometry,features_id,preds
218053,"POLYGON ((3378400.000 652720.000, 3378420.000 ...",249459,1
218067,"POLYGON ((3378260.000 652800.000, 3378270.000 ...",249361,1
218071,"POLYGON ((3378140.000 652640.000, 3378160.000 ...",249360,1
218095,"POLYGON ((3378130.000 652600.000, 3378160.000 ...",249189,1
218455,"POLYGON ((3378120.000 652520.000, 3378150.000 ...",248676,1


# 6) Write results to file

**Geopackage**

In [354]:
path_results_geopackage = '/home/madmex_user/agriculture_geopackages/features_computed/' + features_string + '/'

In [355]:
path_result_geopackage = os.path.join(path_results_geopackage, date_today 
                                       + '/' + dc_tile + '/')

In [356]:
if not os.path.exists(path_result_geopackage):
    os.makedirs(path_result_geopackage)

In [357]:
clusters_filename = path_result_geopackage + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi' + '.gpkg'

In [358]:
layer_name = date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi'

In [359]:
gdf_df_clusters.crs = crs_dict

In [360]:
gdf_df_clusters.to_file(clusters_filename,
                        layer=layer_name,
                        driver='GPKG')

In [361]:
shutil.move(clusters_filename,path_result_dc_tile)

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-23-2020/48_-36/02-23-2020_clusters_48_-36_nclusters_2_aoi2.gpkg'

**Shapefile**

In [362]:
clusters_filename = path_result_dc_tile + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi2' + '.shp'

In [363]:
gdf_df_clusters.to_file(clusters_filename)


In [364]:
clusters_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-23-2020/48_-36/02-23-2020_clusters_48_-36_nclusters_2_aoi2.shp'